# Étape 2 : Feature Engineering

## Objectif
Créer de nouvelles features à partir des données structurelles existantes pour améliorer la prédiction de :
- `SiteEnergyUse(kBtu)` : Consommation énergétique totale
- `TotalGHGEmissions` : Émissions totales de CO2

## Stratégie
Nous allons créer des features couvrant **6 catégories** :
1. **Sources d'énergie** : Diversité et types d'énergie utilisés
2. **Temporalité** : Âge et époque de construction
3. **Structure** : Ratios de surface, hauteur, complexité
4. **Usages** : Diversité et types d'usages
5. **Localisation** : Position géographique
6. **Regroupement** : Réduction de cardinalité

**Note importante** : Les features de sources d'énergie doivent être créées AVANT de supprimer les colonnes de leakage.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Constantes
TARGET_CO2 = 'TotalGHGEmissions'
TARGET_ENERGY = 'SiteEnergyUse(kBtu)'
CIBLES = [TARGET_CO2, TARGET_ENERGY]

# Chargement des données nettoyées de l'étape 1
df_origin = pd.read_csv('data/data_EDA.csv')

df = df_origin.copy()
print(f"📊 Dataset chargé")
print(f"   Shape : {df.shape}")
print(f"   Lignes : {len(df):,} bâtiments")
print(f"   Colonnes : {df.shape[1]}")

📊 Dataset chargé
   Shape : (1630, 38)
   Lignes : 1,630 bâtiments
   Colonnes : 38


## 1. Features liées aux SOURCES D'ÉNERGIE

### Objectif
Créer des features indiquant **quels types de sources d'énergie** sont utilisés par chaque bâtiment.

### Pourquoi c'est important ?
- Le **type d'énergie** influence directement les **émissions de CO2**
- Gaz naturel émet ~5x plus de CO2 que l'électricité (hydroélectrique à Seattle)
- La **diversité énergétique** indique la complexité du bâtiment
- Un bâtiment multi-sources a généralement une consommation plus élevée

### ⚠️ ATTENTION AU DATA LEAKAGE
- ❌ On NE PEUT PAS utiliser les **valeurs** de consommation (`SteamUse(kBtu)`, `Electricity(kBtu)`, `NaturalGas(kBtu)`)
- ✅ On PEUT utiliser l'**existence** de ces sources (présence/absence)
- Ces informations structurelles sont **indépendantes** de l'intensité de consommation

In [2]:
print("\n" + "="*80)
print("🔋 CRÉATION DES FEATURES DE SOURCES D'ÉNERGIE")
print("="*80)

# Feature 1 : Présence de chauffage vapeur
df['has_steam'] = (df['SteamUse(kBtu)'] > 0).astype(int)
print(f"✅ has_steam : {df['has_steam'].sum()} bâtiments avec vapeur ({df['has_steam'].mean()*100:.1f}%)")

# Feature 2 : Présence d'électricité
df['has_electricity'] = (df['Electricity(kBtu)'] > 0).astype(int)
print(f"✅ has_electricity : {df['has_electricity'].sum()} bâtiments avec électricité ({df['has_electricity'].mean()*100:.1f}%)")

# Feature 3 : Présence de gaz naturel
df['has_natural_gas'] = (df['NaturalGas(kBtu)'] > 0).astype(int)
print(f"✅ has_natural_gas : {df['has_natural_gas'].sum()} bâtiments avec gaz ({df['has_natural_gas'].mean()*100:.1f}%)")

# Feature 4 : Diversité des sources d'énergie (0 à 3)
df['energy_source_diversity'] = (
    df['has_steam'] + 
    df['has_electricity'] + 
    df['has_natural_gas']
)
print(f"\n✅ energy_source_diversity :")
print(df['energy_source_diversity'].value_counts().sort_index())

# Feature 5 : Indicateur multi-énergie (2+ sources)
df['is_multi_energy'] = (df['energy_source_diversity'] >= 2).astype(int)
print(f"\n✅ is_multi_energy : {df['is_multi_energy'].sum()} bâtiments multi-énergies ({df['is_multi_energy'].mean()*100:.1f}%)")

print(f"\n📊 Résumé : 5 features créées pour les sources d'énergie")


🔋 CRÉATION DES FEATURES DE SOURCES D'ÉNERGIE
✅ has_steam : 114 bâtiments avec vapeur (7.0%)
✅ has_electricity : 1630 bâtiments avec électricité (100.0%)
✅ has_natural_gas : 1177 bâtiments avec gaz (72.2%)

✅ energy_source_diversity :
energy_source_diversity
1     402
2    1165
3      63
Name: count, dtype: int64

✅ is_multi_energy : 1228 bâtiments multi-énergies (75.3%)

📊 Résumé : 5 features créées pour les sources d'énergie


## 2. Features liées à la TEMPORALITÉ

### Objectif
Capturer l'impact de l'**âge du bâtiment** sur sa consommation énergétique.

### Pourquoi c'est important ?
- Chaque **décennie** a ses propres standards de construction
- L'âge est plus parlant qu'une année brute pour un modèle ML

In [3]:
print("\n" + "="*80)
print("📅 CRÉATION DES FEATURES DE TEMPORALITÉ")
print("="*80)

# Année de référence : année des relevés (2016)
REFERENCE_YEAR = 2016

# Feature 6 : Âge du bâtiment (en années depuis 2016)
df['building_age'] = REFERENCE_YEAR - df['YearBuilt']
print(f"✅ building_age : min={df['building_age'].min()}, max={df['building_age'].max()}, moyenne={df['building_age'].mean():.1f} ans")

# Feature : Classes larges d'âge
df["building_age_bucket"] = pd.cut(
    df["building_age"],
    bins=[-np.inf, 20, 50, np.inf],
    labels=["recent", "intermediate", "old"]
)

# Feature 7 : Décennie de construction
df['decade_built'] = ((df['YearBuilt'] // 10) * 10).astype(str)
print(f"\n✅ decade_built : {df['decade_built'].nunique()} décennies représentées")
print(df['decade_built'].value_counts().sort_index().tail(10))

# Feature 8 : Bâtiment ancien (> 50 ans)
df['is_old_building'] = (df['building_age'] > 50).astype(int)
print(f"\n✅ is_old_building : {df['is_old_building'].sum()} bâtiments anciens ({df['is_old_building'].mean()*100:.1f}%)")

# Feature 10 : Bâtiment récent (< 20 ans, post-1996)
df['is_recent_building'] = (df['building_age'] < 20).astype(int)
print(f"✅ is_recent_building : {df['is_recent_building'].sum()} bâtiments récents ({df['is_recent_building'].mean()*100:.1f}%)")

print(f"\n📊 Résumé : 5 features créées pour la temporalité")


📅 CRÉATION DES FEATURES DE TEMPORALITÉ
✅ building_age : min=1, max=116, moyenne=54.0 ans

✅ decade_built : 12 décennies représentées
decade_built
1920    168
1930     54
1940     63
1950    161
1960    225
1970    164
1980    168
1990    147
2000    197
2010     56
Name: count, dtype: int64

✅ is_old_building : 814 bâtiments anciens (49.9%)
✅ is_recent_building : 314 bâtiments récents (19.3%)

📊 Résumé : 5 features créées pour la temporalité


## 3. Features liées à la STRUCTURE

### Objectif
Créer des **ratios et indicateurs** sur la composition physique des bâtiments.

### Pourquoi c'est important ?
- **Parking** : Zones souvent non chauffées → ratio parking élevé = consommation réduite par m²
- **Hauteur** : Tours (>10 étages) → ascenseurs, pompes, ventilation verticale → surconsommation
- **Surface/étage** : Bâtiments étalés vs compacts → efficacité thermique différente
- **Campus** : Multi-bâtiments → réseaux de distribution → pertes énergétiques

In [4]:
print("\n" + "="*80)
print("🏗️ CRÉATION DES FEATURES DE STRUCTURE")
print("="*80)

# Feature 11 : Ratio de parking (surface parking / surface totale)
df['parking_ratio'] = df['PropertyGFAParking'] / df['PropertyGFATotal']
df['parking_ratio'] = df['parking_ratio'].fillna(0)  # Si division par 0
print(f"✅ parking_ratio : moyenne={df['parking_ratio'].mean():.3f}, max={df['parking_ratio'].max():.3f}")

# Feature 12 : Présence de parking
df['has_parking'] = (df['PropertyGFAParking'] > 0).astype(int)
print(f"✅ has_parking : {df['has_parking'].sum()} bâtiments avec parking ({df['has_parking'].mean()*100:.1f}%)")

# Feature 13 : Ratio bâtiment (surface bâtiment / surface totale)
df['building_ratio'] = df['PropertyGFABuilding(s)'] / df['PropertyGFATotal']
df['building_ratio'] = df['building_ratio'].fillna(1)  # Si pas de parking, ratio = 1
print(f"✅ building_ratio : moyenne={df['building_ratio'].mean():.3f}")

# Feature 14 : Surface par étage (GFA bâtiment / nombre étages)
df['gfa_per_floor'] = df['PropertyGFABuilding(s)'] / df['NumberofFloors']
df['gfa_per_floor'] = df['gfa_per_floor'].replace([np.inf, -np.inf], np.nan).fillna(df['PropertyGFABuilding(s)'])
print(f"✅ gfa_per_floor : moyenne={df['gfa_per_floor'].mean():.0f} sqft/étage")

# Feature 15 : Étages par bâtiment
df['floor_per_building'] = df['NumberofFloors'] / df['NumberofBuildings']
df['floor_per_building'] = df['floor_per_building'].replace([np.inf, -np.inf], np.nan).fillna(df['NumberofFloors'])
print(f"✅ floor_per_building : moyenne={df['floor_per_building'].mean():.1f} étages/bâtiment")

# Feature 16 : Gratte-ciel (> 10 étages)
df['is_tall_building'] = (df['NumberofFloors'] > 10).astype(int)
print(f"✅ is_tall_building : {df['is_tall_building'].sum()} gratte-ciels ({df['is_tall_building'].mean()*100:.1f}%)")

# Feature 17 : Grand bâtiment (> 100,000 sqft)
df['is_large_building'] = (df['PropertyGFATotal'] > 100000).astype(int)
print(f"✅ is_large_building : {df['is_large_building'].sum()} grands bâtiments ({df['is_large_building'].mean()*100:.1f}%)")

# Feature 18 : Campus (plusieurs bâtiments)
df['is_campus'] = (df['NumberofBuildings'] > 1).astype(int)
print(f"✅ is_campus : {df['is_campus'].sum()} campus ({df['is_campus'].mean()*100:.1f}%)")

print(f"\n📊 Résumé : 8 features créées pour la structure")


🏗️ CRÉATION DES FEATURES DE STRUCTURE
✅ parking_ratio : moyenne=0.062, max=0.895
✅ has_parking : 331 bâtiments avec parking (20.3%)
✅ building_ratio : moyenne=0.938
✅ gfa_per_floor : moyenne=35367 sqft/étage
✅ floor_per_building : moyenne=4.1 étages/bâtiment
✅ is_tall_building : 111 gratte-ciels (6.8%)
✅ is_large_building : 433 grands bâtiments (26.6%)
✅ is_campus : 52 campus (3.2%)

📊 Résumé : 8 features créées pour la structure


## 4. Features liées aux USAGES

### Objectif
Capturer la **diversité et les types d'usages** des bâtiments.

### Pourquoi c'est important ?
- **Restaurant** : Cuisines + réfrigération → 3-5x plus énergivore qu'un bureau
- **Data center** : Serveurs 24/7 + climatisation intensive → 10-20x plus énergivore
- **Usages multiples** : Horaires étendus, besoins variés → consommation complexe
- **Dominance d'usage** : Bâtiment spécialisé (90% bureaux) vs mixte (40% bureaux + 30% restaurant)

In [5]:
print("\n" + "="*80)
print("🏢 CRÉATION DES FEATURES D'USAGES")
print("="*80)

# Feature 19 : Usage multiple (présence de virgule dans la liste)
df['has_multiple_uses'] = df['ListOfAllPropertyUseTypes'].str.contains(',', na=False).astype(int)
print(f"✅ has_multiple_uses : {df['has_multiple_uses'].sum()} bâtiments multi-usages ({df['has_multiple_uses'].mean()*100:.1f}%)")

# Feature 20 : Nombre d'usages différents
df['use_count'] = df['ListOfAllPropertyUseTypes'].str.count(',') + 1
print(f"\n✅ use_count :")
print(df['use_count'].value_counts().sort_index())

# Feature 21 : Présence d'un restaurant
df['has_restaurant'] = df['ListOfAllPropertyUseTypes'].str.contains('Restaurant|Food', case=False, na=False).astype(int)
print(f"\n✅ has_restaurant : {df['has_restaurant'].sum()} bâtiments avec restaurant ({df['has_restaurant'].mean()*100:.1f}%)")

# Feature 22 : Présence de commerce/retail
df['has_retail'] = df['ListOfAllPropertyUseTypes'].str.contains('Retail|Store', case=False, na=False).astype(int)
print(f"✅ has_retail : {df['has_retail'].sum()} bâtiments avec commerce ({df['has_retail'].mean()*100:.1f}%)")

# Feature 23 : Présence de data center
df['has_data_center'] = df['ListOfAllPropertyUseTypes'].str.contains('Data Center', case=False, na=False).astype(int)
print(f"✅ has_data_center : {df['has_data_center'].sum()} bâtiments avec data center ({df['has_data_center'].mean()*100:.1f}%)")

# Feature 24 : Ratio de l'usage principal (dominance)
df['largest_use_ratio'] = df['LargestPropertyUseTypeGFA'] / df['PropertyGFATotal']
df['largest_use_ratio'] = df['largest_use_ratio'].fillna(1)
print(f"\n✅ largest_use_ratio : moyenne={df['largest_use_ratio'].mean():.3f} (1 = mono-usage)")

print(f"\n📊 Résumé : 6 features créées pour les usages")


🏢 CRÉATION DES FEATURES D'USAGES
✅ has_multiple_uses : 848 bâtiments multi-usages (52.0%)

✅ use_count :
use_count
1     782
2     494
3     203
4      81
5      41
6      19
7       4
8       1
9       3
11      1
13      1
Name: count, dtype: int64

✅ has_restaurant : 133 bâtiments avec restaurant (8.2%)
✅ has_retail : 300 bâtiments avec commerce (18.4%)
✅ has_data_center : 43 bâtiments avec data center (2.6%)

✅ largest_use_ratio : moyenne=0.869 (1 = mono-usage)

📊 Résumé : 6 features créées pour les usages


## 5. Features liées à la LOCALISATION

### Objectif
Capturer l'impact de la **position géographique** sur la consommation.

### Pourquoi c'est important ?
- **Centre-ville** : Bâtiments denses, anciens, gratte-ciels → forte consommation
- **Périphérie** : Bâtiments récents, étalés, moins denses → consommation modérée
- **Zones industrielles** : Usages spécifiques (entrepôts, production)
- **Distance au centre** : Proxy de l'âge et du type urbain

In [6]:
print("\n" + "="*80)
print("📍 CRÉATION DES FEATURES DE LOCALISATION")
print("="*80)

# Centre approximatif de Seattle (Downtown)
CENTER_LAT = 47.6062
CENTER_LON = -122.3321

# Feature 25 : Distance au centre (en degrés, approximation)
df['distance_to_center'] = np.sqrt(
    (df['Latitude'] - CENTER_LAT)**2 + 
    (df['Longitude'] - CENTER_LON)**2
)
print(f"✅ distance_to_center : moyenne={df['distance_to_center'].mean():.4f}°, max={df['distance_to_center'].max():.4f}°")

# Feature 26 : Bâtiment du centre-ville
df['is_downtown'] = (df['Neighborhood'] == 'DOWNTOWN').astype(int)
print(f"✅ is_downtown : {df['is_downtown'].sum()} bâtiments downtown ({df['is_downtown'].mean()*100:.1f}%)")

# Feature 27 : Zone industrielle
industrial_zones = ['GREATER DUWAMISH', 'BALLARD', 'INTERBAY']
df['is_industrial_area'] = df['Neighborhood'].isin(industrial_zones).astype(int)
print(f"✅ is_industrial_area : {df['is_industrial_area'].sum()} bâtiments en zone industrielle ({df['is_industrial_area'].mean()*100:.1f}%)")

print(f"\n📊 Résumé : 3 features créées pour la localisation")


📍 CRÉATION DES FEATURES DE LOCALISATION
✅ distance_to_center : moyenne=0.0439°, max=0.1298°
✅ is_downtown : 353 bâtiments downtown (21.7%)
✅ is_industrial_area : 410 bâtiments en zone industrielle (25.2%)

📊 Résumé : 3 features créées pour la localisation


## 7. Bilan du feature engineering

### Résumé des features créées

In [7]:
print("\n" + "="*80)
print("📊 BILAN COMPLET DU FEATURE ENGINEERING")
print("="*80)

# Liste de toutes les nouvelles features créées
new_features = [
    # Catégorie 1 : Sources d'énergie (5)
    'has_steam', 'has_natural_gas', 'energy_source_diversity', 'is_multi_energy',
    
    # Catégorie 2 : Temporalité (5)
    'building_age', 'decade_built', 'is_old_building', 'is_recent_building',
    
    # Catégorie 3 : Structure (8)
    'parking_ratio', 'has_parking', 'building_ratio', 'gfa_per_floor', 'floor_per_building',
    'is_tall_building', 'is_large_building', 'is_campus',
    
    # Catégorie 4 : Usages (6)
    'has_multiple_uses', 'use_count', 'has_restaurant', 'has_retail', 'has_data_center', 'largest_use_ratio',
    
    # Catégorie 5 : Localisation (3)
    'distance_to_center', 'is_downtown', 'is_industrial_area'
]

print(f"\n✅ TOTAL : {len(new_features)} nouvelles features créées\n")

categories = {
    '🔋 Sources d\'énergie': 5,
    '📅 Temporalité': 5,
    '🏗️ Structure': 8,
    '🏢 Usages': 6,
    '📍 Localisation': 3,
    '🗂️ Regroupement': 2,
    '🔗 Interaction': 4
}

for category, count in categories.items():
    print(f"{category:30s} : {count:2d} features")

print(f"\n{'='*80}")
print(f"Shape finale du dataset : {df.shape}")
print(f"  • Lignes : {len(df):,} bâtiments")
print(f"  • Colonnes : {df.shape[1]} (dont {len(new_features)} nouvelles features)")
print(f"{'='*80}")


📊 BILAN COMPLET DU FEATURE ENGINEERING

✅ TOTAL : 25 nouvelles features créées

🔋 Sources d'énergie            :  5 features
📅 Temporalité                  :  5 features
🏗️ Structure                   :  8 features
🏢 Usages                       :  6 features
📍 Localisation                 :  3 features
🗂️ Regroupement                :  2 features
🔗 Interaction                  :  4 features

Shape finale du dataset : (1630, 65)
  • Lignes : 1,630 bâtiments
  • Colonnes : 65 (dont 25 nouvelles features)


## 7. Vérification de la qualité des features

Vérifions qu'il n'y a pas de valeurs aberrantes ou de problèmes dans nos nouvelles features.

In [8]:
print("\n" + "="*80)
print("🔍 VÉRIFICATION DE LA QUALITÉ DES FEATURES")
print("="*80)

# Vérifier les valeurs manquantes dans les nouvelles features
print("\n1️⃣ Valeurs manquantes dans les nouvelles features :")
missing_in_new_features = df[new_features].isnull().sum()
if missing_in_new_features.sum() == 0:
    print("   ✅ Aucune valeur manquante !")
else:
    print(missing_in_new_features[missing_in_new_features > 0])

# Vérifier les valeurs infinies
print("\n2️⃣ Valeurs infinies dans les nouvelles features :")
numeric_new_features = df[new_features].select_dtypes(include=[np.number]).columns
inf_count = np.isinf(df[numeric_new_features]).sum()
if inf_count.sum() == 0:
    print("   ✅ Aucune valeur infinie !")
else:
    print(inf_count[inf_count > 0])

# Statistiques descriptives des features numériques
print("\n3️⃣ Statistiques des features numériques créées :")
display(df[numeric_new_features].describe().round(2))


🔍 VÉRIFICATION DE LA QUALITÉ DES FEATURES

1️⃣ Valeurs manquantes dans les nouvelles features :
   ✅ Aucune valeur manquante !

2️⃣ Valeurs infinies dans les nouvelles features :
   ✅ Aucune valeur infinie !

3️⃣ Statistiques des features numériques créées :


,has_steam,has_natural_gas,energy_source_diversity,is_multi_energy,building_age,is_old_building,is_recent_building,parking_ratio,has_parking,building_ratio,gfa_per_floor,floor_per_building,is_tall_building,is_large_building,is_campus,has_multiple_uses,use_count,has_restaurant,has_retail,has_data_center,largest_use_ratio,distance_to_center,is_downtown,is_industrial_area
count,1630.00,1630.00,1630.00,1630.00,1630.00,1630.0,1630.00,1630.00,1630.0,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00,1630.00
mean,0.07,0.72,1.79,0.75,53.96,0.5,0.19,0.06,0.2,0.94,35366.97,4.06,0.07,0.27,0.03,0.52,1.91,0.08,0.18,0.03,0.87,0.04,0.22,0.25
std,0.26,0.45,0.49,0.43,32.66,0.5,0.39,0.14,0.4,0.14,124930.02,6.20,0.25,0.44,0.18,0.50,1.23,0.27,0.39,0.16,0.31,0.03,0.41,0.43
min,0.00,0.00,1.00,0.00,1.00,0.0,0.00,0.00,0.0,0.10,1818.00,0.02,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.19,0.00,0.00,0.00
25%,0.00,0.00,2.00,1.00,27.00,0.0,0.00,0.00,0.0,1.00,12302.69,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.69,0.02,0.00,0.00
50%,0.00,1.00,2.00,1.00,50.00,0.0,0.00,0.00,0.0,1.00,21005.50,2.00,0.00,0.00,0.00,1.00,2.00,0.00,0.00,0.00,0.96,0.04,0.00,0.00
75%,0.00,1.00,2.00,1.00,86.00,1.0,0.00,0.00,0.0,1.00,33700.88,4.00,0.00,1.00,0.00,1.00,2.00,0.00,0.00,0.00,1.00,0.07,0.00,1.00
max,1.00,1.00,3.00,1.00,116.00,1.0,1.00,0.90,1.0,1.00,4660078.00,76.00,1.00,1.00,1.00,1.00,13.00,1.00,1.00,1.00,6.43,0.13,1.00,1.00


## 8. Sauvegarde du dataset enrichi

Nous sauvegardons le dataset avec toutes les nouvelles features créées.

⚠️ **Important** : Nous conservons encore les colonnes de leakage pour le moment (elles seront supprimées à l'étape 3).

In [9]:
print("\n" + "="*80)
print("💾 SAUVEGARDE DU DATASET ENRICHI")
print("="*80)

# Sauvegarder le dataset
output_path = 'data/data_feature_engineering.csv'
df.to_csv(output_path, index=False)

print(f"\n✅ Dataset sauvegardé : {output_path}")
print(f"   Shape : {df.shape}")
print(f"   Taille : {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n" + "="*80)
print("🎉 ÉTAPE 2 - FEATURE ENGINEERING TERMINÉE !")
print("="*80)
print(f"\n📦 {len(new_features)} nouvelles features créées")
print(f"📊 {len(df)} bâtiments enrichis")
print(f"✅ Dataset prêt pour l'étape 3 (Préparation pour modélisation)")
print("\n💡 Prochaine étape :")
print("   - Supprimer les colonnes de leakage")
print("   - Gérer les outliers")
print("   - Encoder les variables catégorielles")
print("   - Normaliser les features numériques")


💾 SAUVEGARDE DU DATASET ENRICHI

✅ Dataset sauvegardé : data/data_feature_engineering.csv
   Shape : (1630, 65)
   Taille : 1.78 MB

🎉 ÉTAPE 2 - FEATURE ENGINEERING TERMINÉE !

📦 25 nouvelles features créées
📊 1630 bâtiments enrichis
✅ Dataset prêt pour l'étape 3 (Préparation pour modélisation)

💡 Prochaine étape :
   - Supprimer les colonnes de leakage
   - Gérer les outliers
   - Encoder les variables catégorielles
   - Normaliser les features numériques
